In [343]:
#Import libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rapidfuzz
import difflib
from nltk.corpus import stopwords
from yake import KeywordExtractor

In [344]:
Stops = stopwords.words('english') #Returns all stopwords in the English language. 
print(Stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Conneticut

In [345]:
State = 'CT'
AllOccs = pd.read_csv('../../../Data/Post1790/aggregated_CD_noname.csv') #Returns aggregated CSV file. 
AllOccs.drop(columns=['Unnamed: 0'], inplace=True) #Drop duplicate index column which appears.
AllOccsCT = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsCT = AllOccsCT.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsCT)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       1064.0      75.0          532.0          37.0      508.0      51.0   
1        449.0      96.0          224.0          97.0      232.0      10.0   
2        196.0      75.0           98.0          37.0      172.0      24.0   
3         53.0      58.0           26.0          79.0       67.0       6.0   
4        121.0      22.0           60.0          61.0       32.0      72.0   
..         ...       ...            ...           ...        ...       ...   
478       39.0      11.0           19.0          55.0       14.0       7.0   
479       13.0      21.0            6.0          61.0        4.0      74.0   
480      533.0      34.0          266.0          66.0      446.0      66.0   
481      658.0      47.0          329.0          23.0      558.0      50.0   
482       12.0      22.0            6.0          11.0       11.0       NaN   

           town state occupation             county name_type  

In [346]:
OccsTotalsCT = AllOccsCT.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsCT.to_string())

                                                           occupation  6p_total
0                                   Adminastrator of the Asa Waterman    218.47
1                                      Adminastrator to David Fellows      0.00
2   Adminastrator to the Estate of Epaphras Bull of Hartford Deceased     61.30
3                        Adminastrator to the Estate of Samuel Tilley     94.67
4            Adminastrator to the estate of Uriah Loomis Jun Deceased     73.50
5                                          Administrator to J McCurdy   3413.72
6                         Administrator to the estate of Daniel Unkus     54.05
7                Adminstrator to the Estate of Ezekiel Carey Deceased     77.61
8                  Adminstrator to the Estate of Ruth Skiffe Deceased     87.74
9                                                          Apothecary      7.72
10                                                           Attorney   4863.67
11                                      

In [347]:
#Function that returns the same dataframe, but with all stopwords removed from the occupation titles. 
def RemoveStopwords(OccsTotals):
    OccsTotals['occupation'] = OccsTotals['occupation'].apply(lambda OccName: ' '.join(word.lower() for word in OccName.split() if word not in Stops))
    return OccsTotals 

In [348]:
OccsTotalsCT = RemoveStopwords(OccsTotalsCT) #Call 
print(OccsTotalsCT.to_string())

                                              occupation  6p_total
0                             adminastrator asa waterman    218.47
1                            adminastrator david fellows      0.00
2   adminastrator estate epaphras bull hartford deceased     61.30
3                     adminastrator estate samuel tilley     94.67
4         adminastrator estate uriah loomis jun deceased     73.50
5                                administrator j mccurdy   3413.72
6                      administrator estate daniel unkus     54.05
7             adminstrator estate ezekiel carey deceased     77.61
8               adminstrator estate ruth skiffe deceased     87.74
9                                             apothecary      7.72
10                                              attorney   4863.67
11                                          attorney law   6250.93
12                                            blacksmith    724.71
13                                                broker    20

In [349]:
#This function will help determine how many words we can acceptably cut off from the end. 
def FindMaxLength(Occupations):
    TempMaxLength = 2
    
    print('Occupation Before --> Occupation After')
    print('--------------------------------------')
    print('\n')
    
    for Occupation in Occupations:
        if (len(Occupation.split()) > TempMaxLength): #If occupation has more than two words, remove last two words.
            OccupationAfter = Occupation.rsplit(' ', 2)[0] 
        else:
            OccupationAfter = Occupation #Else, set them both equal to each other; nothing changes. 
        
        print(Occupation + ' --> ' + OccupationAfter)

In [350]:
FindMaxLength(OccsTotalsCT['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


adminastrator asa waterman --> adminastrator
adminastrator david fellows --> adminastrator
adminastrator estate epaphras bull hartford deceased --> adminastrator estate epaphras bull
adminastrator estate samuel tilley --> adminastrator estate
adminastrator estate uriah loomis jun deceased --> adminastrator estate uriah loomis
administrator j mccurdy --> administrator
administrator estate daniel unkus --> administrator estate
adminstrator estate ezekiel carey deceased --> adminstrator estate ezekiel
adminstrator estate ruth skiffe deceased --> adminstrator estate ruth
apothecary --> apothecary
attorney --> attorney
attorney law --> attorney law
blacksmith --> blacksmith
broker --> broker
carpenter --> carpenter
cashier hartford bank --> cashier
clothier --> clothier
committee 1st ecclestical society --> committee 1st
committee 1st society plainville --> committee 1st
cooper --> cooper
doctor --> doctor
esqui

In [351]:
#Declare necessary parameters for KeywordExtractor() function.
MaxLength = 2
language = 'en' #Set language to English.
MaxSize = 1  #Each keyword can only be a max. length of 1.
DupThres = 0.5 #Words can not be used repeatedly in keywords. Treat each word separately. 
KeyExtractor = KeywordExtractor(lan=language, n=MaxSize, dedupLim=DupThres, features=None) #Create KeywordExtractor object.

In [352]:
#Print what the merged occupations will look like. This will help us find any exceptions. 
def PrintFutureOccupations(Occupations):
    print('Occupations --> Potential Cleaned Occupations')
    print('---------------------------------------------')
    print('\n')
        
    for Occupation in Occupations:
        if (len(Occupation.split()) > MaxLength): #If occupation has more than two words in it, remove last two words. Depends on constant. Might change. 
            Occupation = Occupation.rsplit(' ', 2)[0]
            
        Keywords = KeyExtractor.extract_keywords(Occupation) #Retrieve list of keywords. 
        
        print(Occupation + ' --> ' + str(Keywords[0][0])) #Print current occupation title to hypothetical cleaned occupation title.

In [353]:
PrintFutureOccupations(OccsTotalsCT['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


adminastrator --> adminastrator
adminastrator --> adminastrator
adminastrator estate epaphras bull --> adminastrator
adminastrator estate --> adminastrator
adminastrator estate uriah loomis --> adminastrator
administrator --> administrator
administrator estate --> administrator
adminstrator estate ezekiel --> adminstrator
adminstrator estate ruth --> adminstrator
apothecary --> apothecary
attorney --> attorney
attorney law --> attorney
blacksmith --> blacksmith
broker --> broker
carpenter --> carpenter
cashier --> cashier
clothier --> clothier
committee 1st --> committee
committee 1st --> committee
cooper --> cooper
doctor --> doctor
esquire --> esquire
executor ezra selden --> executor
executor stephen --> executor
executor stephen --> executor
executor william --> executor
executor estate tim --> executor
executor estate uriah --> executor
executors estate jabez --> executors
farmer --> farm

In [354]:
ExceptionsCT = {'inn keeper' : 'inn keeper', 'leather dresser' : 'leather dresser', 'post rider' : 'post rider', 
                'school committee' : 'school committee', 'school master' : 'school master', 'society committee' : 'committee',
               'society treasurer' : 'treasurer'}

In [355]:
#Handle duplicate occupations that are spelled differently. Pass in 'occupation' column as a list. Pass in function which will do a manual check for each state (differs w/ each state). 
def MergeOccupations(OccsTotals, Exceptions):
    Occupations = OccsTotals['occupation'].tolist() #Returns 'occupation' column as a list.    
    Index = 0 #Keep track of index. 
    
    for Occupation in Occupations:
        if (len(Occupation.split()) > MaxLength): #If occupation has more than two words in it, remove last two words. Depends on constant. Might change. 
            Occupation = Occupation.rsplit(' ', 2)[0]
            
        Keywords = KeyExtractor.extract_keywords(Occupation) #Retrieve list of keywords. 
        NewOccupation = Keywords[0][0] #Get new occupation from list of keywords. Based on which keyword has the highest rating.
        
        if (Occupation in Exceptions): #If occupation is an exception, set new occupation. 
            NewOccupation = Exceptions[Occupation]
    
        OccsTotals.loc[Index, 'occupation'] = NewOccupation #Replace occupation with new occupation in dataframe.
        
        Index += 1 #Increment. 
    
    print('After Cleaning')
    print('--------------')
    print('\n')
    print(OccsTotals.to_string()) #Check
    print('\n')
    
    OccsTotals = OccsTotals.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations, reset index. 
    
    print('After Removing Duplicates and Summing')
    print('-------------------------------------')
    print('\n')
    print(OccsTotals.to_string()) #Check
    print('\n')
    
    Occupations = OccsTotals['occupation'].tolist() #Reinstantiate Occupations after changing OccsTotals
    i = 0 #Declare new index.
    while (i < len(Occupations)):
        j = 0
        while (j < len(Occupations)):
            if (rapidfuzz.fuzz.partial_ratio(Occupations[i], Occupations[j]) > 80) and (Occupations[i] != Occupations[j]):                
                Total = float(OccsTotals.loc[j, '6p_total'])
                OccsTotals.loc[i, '6p_total'] += Total 
                OccsTotals.drop(j, inplace=True)
                OccsTotals = OccsTotals.reset_index(drop=True)
                Occupations.remove(Occupations[j])
            else:
                j += 1
                
        i += 1
    
    print('\n')
    print('After Using Fuzzy String Matching')
    print('---------------------------------')
    print('\n')
    print(OccsTotals.to_string())
    print('\n')
    
    return OccsTotals
        

In [356]:
OccsTotalsCT = MergeOccupations(OccsTotalsCT, ExceptionsCT)

After Cleaning
--------------


          occupation  6p_total
0      adminastrator    218.47
1      adminastrator      0.00
2      adminastrator     61.30
3      adminastrator     94.67
4      adminastrator     73.50
5      administrator   3413.72
6      administrator     54.05
7       adminstrator     77.61
8       adminstrator     87.74
9         apothecary      7.72
10          attorney   4863.67
11          attorney   6250.93
12        blacksmith    724.71
13            broker    208.55
14         carpenter    175.34
15           cashier     85.30
16          clothier     53.33
17         committee    306.28
18         committee    220.49
19            cooper    124.76
20            doctor    647.09
21           esquire  13020.09
22          executor   1963.62
23          executor      6.02
24          executor    266.67
25          executor      4.70
26          executor   1573.33
27          executor    614.19
28         executors    129.72
29            farmer  36715.82
30     

In [357]:
OccsTotalsCT.to_excel("./FinishedSpreadsheets/Occupations" + state + ".xlsx")